In [5]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [7]:
openai.api_key  = os.environ['OPENAI_API_KEY']

KeyError: 'OPENAI_API_KEY'

In [9]:
!echo $OPENAI_API_KEY